In [20]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, MultiPolygon
from tqdm import tqdm

In [3]:
dissemination_area_file = "data/statscan/census_2021/dissemination_areas/lda_000a21a_e/lda_000a21a_e.shp"
census_file = "data/statscan/census_2021/98-401-X2021006_Quebec_eng_CSV/98-401-X2021006_English_CSV_data_Quebec.csv"
census_encoding = "ISO-8859-1"

In [4]:
# Read dissemination area file
dissemination_areas = gpd.read_file(dissemination_area_file)
print(dissemination_areas.head())

      DAUID              DGUID  LANDAREA PRUID  \
0  10010232  2021S051210010232    0.0759    10   
1  10010233  2021S051210010233    0.1246    10   
2  10010234  2021S051210010234    0.1031    10   
3  10010235  2021S051210010235    0.0846    10   
4  10010236  2021S051210010236    0.1055    10   

                                            geometry  
0  POLYGON ((8979060.777 2152103.386, 8979061.674...  
1  POLYGON ((8978740.609 2151923.786, 8978723.483...  
2  POLYGON ((8979010.177 2152087.183, 8979035.303...  
3  POLYGON ((8978535.709 2151470.283, 8978455.289...  
4  POLYGON ((8978375.923 2151363.186, 8978285.711...  


In [5]:
# Read census file (~6 GB)
census = pd.read_csv(census_file, encoding=census_encoding)
print(census.head())

/var/folders/nz/8f1znby90q17dz1y25g4208h0000gn/T/ipykernel_95435/2996579066.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  census = pd.read_csv(census_file, encoding=census_encoding)


   CENSUS_YEAR           DGUID  ALT_GEO_CODE GEO_LEVEL GEO_NAME  TNR_SF  \
0         2021  2021A000011124             1   Country   Canada     3.1   
1         2021  2021A000011124             1   Country   Canada     3.1   
2         2021  2021A000011124             1   Country   Canada     3.1   
3         2021  2021A000011124             1   Country   Canada     3.1   
4         2021  2021A000011124             1   Country   Canada     3.1   

   TNR_LF  DATA_QUALITY_FLAG  CHARACTERISTIC_ID  \
0     4.3              20000                  1   
1     4.3              20000                  2   
2     4.3              20000                  3   
3     4.3              20000                  4   
4     4.3              20000                  5   

                             CHARACTERISTIC_NAME  ...  C2_COUNT_MEN+  \
0                               Population, 2021  ...            NaN   
1                               Population, 2016  ...            NaN   
2     Population percentag

In [6]:
dissemination_areas_dguids = dissemination_areas["DGUID"]
census_dguids = census["DGUID"]

In [7]:
common_dguids = set(dissemination_areas_dguids) & set(census_dguids)
print(len(common_dguids))
print(list(common_dguids)[:5])

13806
['2021S051224960055', '2021S051224650468', '2021S051224720169', '2021S051224940018', '2021S051224230113']


In [8]:
census_qc = census[census["DGUID"].isin(common_dguids)]
dissemination_areas_qc = dissemination_areas[dissemination_areas["DGUID"].isin(common_dguids)]

print(f"census_qc: {len(census_qc)}/{len(census)} ({len(census_qc)/len(census)}%)")
print(f"dissemination_areas_qc: {len(dissemination_areas_qc)}/{len(dissemination_areas)} ({len(dissemination_areas_qc)/len(dissemination_areas)}%)")

census_qc: 36323586/39959628 (0.9090071108770081%)
dissemination_areas_qc: 13806/57936 (0.23829743164871584%)


# Build a new census geospatial dataset for Quebec
Use dissemination areas as spatial unit.

In [10]:
def combine_hierarchical_rows(df, characteristics_of_interest, indent_column='CHARACTERISTIC_NAME'):
    # Create a copy of the dataframe
    df = df.copy()
    
    # Find indentation level for each row
    df['indent_level'] = df[indent_column].str.len() - df[indent_column].str.lstrip().str.len()
    
    result_rows = []
    
    for characteristic in characteristics_of_interest:
        # Find each occurrence of the characteristic
        char_indices = df[df[indent_column].str.strip() == characteristic].index
        
        for char_idx in char_indices:
            # Add the characteristic row itself
            current_row = df.loc[char_idx].copy()
            current_row['parent'] = None  # or '' if you prefer
            current_row['full_hierarchy'] = characteristic
            result_rows.append(current_row)
            
            # Get the base indentation level
            base_level = df.loc[char_idx, 'indent_level']
            hierarchy_stack = []
            
            # Get all rows that come after this characteristic
            subsequent_rows = df.loc[char_idx + 1:]
            
            # Keep rows until we hit another row with same or lower indentation
            for idx, row in subsequent_rows.iterrows():
                if row['indent_level'] <= base_level:
                    break
                
                # Update hierarchy stack based on indentation
                while hierarchy_stack and row['indent_level'] <= df.loc[hierarchy_stack[-1], 'indent_level']:
                    hierarchy_stack.pop()
                    
                # Add row to results with hierarchy information
                current_row = row.copy()
                current_row['parent'] = characteristic if not hierarchy_stack else df.loc[hierarchy_stack[-1], indent_column].strip()
                
                # Build full hierarchy path
                hierarchy_path = [characteristic] + [df.loc[i, indent_column].strip() for i in hierarchy_stack] + [row[indent_column].strip()]
                current_row['full_hierarchy'] = " > ".join(hierarchy_path)
                
                result_rows.append(current_row)
                hierarchy_stack.append(idx)
    
    # Create new dataframe from collected rows
    if result_rows:
        result_df = pd.DataFrame(result_rows)
        result_df = result_df.drop('indent_level', axis=1)
        return result_df
    else:
        return pd.DataFrame(columns=df.columns)

In [11]:
CHARACTERISTICS_OF_INTEREST = [
    "Total - Age groups of the population - 100% data",
    "Population, 2021",
    "Population, 2016",
    "Total private dwellings",
    "Private dwellings occupied by usual residents",
    "Land area in square kilometres",
    "Total - Household after-tax income groups in 2020 for private households - 100% data",
    "Total - Income statistics for private households - 100% data",
    "Unemployment rate",
]

In [12]:
# Pre-process census data using groupby
census_grouped = census_qc.groupby('DGUID').apply(
    lambda x: combine_hierarchical_rows(x, CHARACTERISTICS_OF_INTEREST, indent_column="CHARACTERISTIC_NAME")
)

/var/folders/nz/8f1znby90q17dz1y25g4208h0000gn/T/ipykernel_95435/1659422197.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  census_grouped = census_qc.groupby('DGUID').apply(


In [13]:
census_grouped

CENSUS_YEAR              DGUID  ALT_GEO_CODE  \
DGUID                                                                      
2021S051224010018 65782            2021  2021S051224010018      24010018   
                  65783            2021  2021S051224010018      24010018   
                  65784            2021  2021S051224010018      24010018   
                  65785            2021  2021S051224010018      24010018   
                  65786            2021  2021S051224010018      24010018   
...                                 ...                ...           ...   
2021S051224990267 39783597         2021  2021S051224990267      24990267   
                  39783598         2021  2021S051224990267      24990267   
                  39783599         2021  2021S051224990267      24990267   
                  39783600         2021  2021S051224990267      24990267   
                  39785580         2021  2021S051224990267      24990267   

                                     GEO_LEVEL  GEO_NAME  TNR_SF  TNR_LF  \
DGUID                                                                      
2021S051224010018 65782     Dissemination area  24010018     2.7     1.7   
                  65783     Dissemination area  24010018     2.7     1.7   
                  65784     Dissemination area  24010018     2.7     1.7   
                  65785     Dissemination area  24010018     2.7     1.7   
                  65786     Dissemination area  24010018     2.7     1.7   
...                                        ...       ...     ...     ...   
2021S051224990267 39783597  Dissemination area  24990267    18.9    21.3   
                  39783598  Dissemination area  24990267    18.9    21.3   
                  39783599  Dissemination area  24990267    18.9    21.3   
                  39783600  Dissemination area  24990267    18.9    21.3   
                  39785580  Dissemination area  24990267    18.9    21.3   

                            DATA_QUALITY_FLAG  CHARACTERISTIC_ID  \
DGUID                                                              
2021S051224010018 65782                     0                  8   
                  65783                     0                  9   
                  65784                     0                 10   
                  65785                     0                 11   
                  65786                     0                 12   
...                                       ...                ...   
2021S051224990267 39783597               1020                247   
                  39783598               1020                248   
                  39783599               1020                249   
                  39783600               1020                250   
                  39785580               1020               2230   

                                                          CHARACTERISTIC_NAME  \
DGUID                                                                           
2021S051224010018 65782      Total - Age groups of the population - 100% data   
                  65783                                         0 to 14 years   
                  65784                                          0 to 4 years   
                  65785                                          5 to 9 years   
                  65786                                        10 to 14 years   
...                                                                       ...   
2021S051224990267 39783597        Median after-tax income of one-person ho...   
                  39783598      Total - Income statistics for two-or-more-...   
                  39783599        Median total income of two-or-more-perso...   
                  39783600        Median after-tax income of two-or-more-p...   
                  39785580                                  Unemployment rate   

                            ...  C3_COUNT_WOMEN+  SYMBOL.2 C10_RATE_TOTAL  \
DGUID                       ...   

In [ ]:
# Create a list to store all rows
all_rows = []

# Process each DGUID
for dguid in tqdm(common_dguids):
    census_at_dguid = census_grouped.loc[dguid]
    dissemination_area_at_dguid = dissemination_areas[dissemination_areas["DGUID"]==dguid]

    # Create the new row data
    new_row_data = {
        row["full_hierarchy"]: row["C1_COUNT_TOTAL"]
        for _, row in census_at_dguid.iterrows()
    }
    new_row_data["DGUID"] = dissemination_area_at_dguid["DGUID"].iloc[0]
    new_row_data["geometry"] = dissemination_area_at_dguid["geometry"].iloc[0]
    all_rows.append(new_row_data)

# Create final GeoDataFrame at once
combined_gdf = gpd.GeoDataFrame(all_rows)
combined_gdf.crs = dissemination_areas.crs
# combined_gdf["DGUID"] = combined_gdf["DGUID"].astype(str)

# Save files
combined_gdf.to_parquet(
    "data/output/census_2021_qc_parsed.parquet",
    compression="snappy",
    index=False,
    engine="pyarrow",
)

100%|██████████| 13806/13806 [00:39<00:00, 345.37it/s]


In [15]:
combined_gdf

,Total - Age groups of the population - 100% data,Total - Age groups of the population - 100% data > 0 to 14 years,Total - Age groups of the population - 100% data > 0 to 14 years > 0 to 4 years,Total - Age groups of the population - 100% data > 0 to 14 years > 5 to 9 years,Total - Age groups of the population - 100% data > 0 to 14 years > 10 to 14 years,Total - Age groups of the population - 100% data > 15 to 64 years,Total - Age groups of the population - 100% data > 15 to 64 years > 15 to 19 years,Total - Age groups of the population - 100% data > 15 to 64 years > 20 to 24 years,Total - Age groups of the population - 100% data > 15 to 64 years > 25 to 29 years,Total - Age groups of the population - 100% data > 15 to 64 years > 30 to 34 years,...,Total - Income statistics for private households - 100% data > Median after-tax income of household in 2020 ($),Total - Income statistics for private households - 100% data > Median after-tax income of household in 2020 ($) > Total - Income statistics for one-person private households - 100% data,Total - Income statistics for private households - 100% data > Median after-tax income of household in 2020 ($) > Total - Income statistics for one-person private households - 100% data > Median total income of one-person households in 2020 ($),Total - Income statistics for private households - 100% data > Median after-tax income of household in 2020 ($) > Total - Income statistics for one-person private households - 100% data > Median after-tax income of one-person households in 2020 ($),Total - Income statistics for private households - 100% data > Median after-tax income of household in 2020 ($) > Total - Income statistics for two-or-more-persons private households - 100% data,Total - Income statistics for private households - 100% data > Median after-tax income of household in 2020 ($) > Total - Income statistics for two-or-more-persons private households - 100% data > Median total income of two-or-more-person households in 2020 ($),Total - Income statistics for private households - 100% data > Median after-tax income of household in 2020 ($) > Total - Income statistics for two-or-more-persons private households - 100% data > Median after-tax income of two-or-more-person households in 2020 ($),Unemployment rate,DGUID,geometry
0,510.0,90.0,25.0,35.0,35.0,330.0,35.0,20.0,25.0,20.0,...,71000.0,80.0,46800.0,39200.0,155.0,113000.0,95000.0,7.8,2021S051224960055,"POLYGON ((7885906.431 1770076.289, 7885855.543..."
1,555.0,85.0,25.0,35.0,30.0,315.0,25.0,30.0,30.0,25.0,...,76500.0,25.0,36800.0,34800.0,180.0,94000.0,83000.0,0.0,2021S051224650468,"POLYGON ((7611209.526 1248282.346, 7611222.729..."
2,430.0,80.0,25.0,35.0,25.0,290.0,20.0,10.0,30.0,40.0,...,54000.0,70.0,33600.0,31400.0,125.0,85000.0,75000.0,0.0,2021S051224720169,"POLYGON ((7601567.591 1235319.934, 7601571.891..."
3,560.0,90.0,25.0,30.0,35.0,335.0,20.0,20.0,15.0,25.0,...,71500.0,50.0,38800.0,34800.0,185.0,103000.0,86000.0,3.8,2021S051224940018,"POLYGON ((7700251.586 1608033.497, 7700469.171..."
4,470.0,55.0,20.0,25.0,10.0,290.0,15.0,20.0,25.0,30.0,...,54000.0,95.0,39200.0,34400.0,120.0,87000.0,73500.0,3.9,2021S051224230113,"POLYGON ((7761437.954 1444769.303, 7761401.351..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13801,1290.0,400.0,95.0,135.0,170.0,810.0,70.0,40.0,30.0,85.0,...,112000.0,50.0,52800.0,44400.0,370.0,147000.0,118000.0,3.8,2021S051224230890,"POLYGON ((7759798.583 1449889.754, 7759802.109..."
13802,465.0,80.0,20.0,30.0,25.0,265.0,30.0,20.0,20.0,15.0,...,63600.0,65.0,25600.0,25200.0,135.0,103000.0,87000.0,7.3,2021S051224570017,"POLYGON ((7657661.897 1260286.929, 7657651.866..."
13803,475.0,95.0,25.0,30.0,35.0,305.0,30.0,20.0,30.0,15.0,...,76500.0,55.0,40000.0,35600.0,145.0,131000.0,102000.0,5.3,2021S051224850085,"POLYGON ((7182030.466 1294911.591, 7182146.131..."
13804,275.0,40.0,5.0,15.0,20.0,190.0,25.0,15.0,10.0,20.0,...,82000.0,20.0,NaN,NaN,90.0,108000.0,90000.0,4.3,2021S0512

# Combine with school data
Goal is to associate schools to StatsCan census regions. Ultimately, we want to be able to apply census data to determine socio-economis factors that apply to schools.

School district algo idea:
- Get school lat/lon
- Read stastcan census data in surroundinga area
- Grow outwards, accumulating students until school is full

In [16]:
# # Merge school names, ids and lat/lon into new csv
# a = pd.read_csv("data/school/merged_clustered_data.csv", index_col="Address")
# b = pd.read_csv("data/school/full_addresses_avec_coor.csv", index_col="original_address")
# schools = a.join(b, how="inner", lsuffix="_a", rsuffix="_b")
# schools = schools.set_index("Code")
# schools = schools[["School Name", "lat", "lon", "Full Address", "Code Postal", "Aggregate Dissemination Area Code", "Number of students"]]
# schools.to_csv("data/output/schools_qc_basic.csv")
# print(len(a))
# print(len(b))
# print(len(schools))
# schools

In [ ]:
def match_schools_to_diss_areas(
        schools_gdf: gpd.GeoDataFrame,
        diss_areas_gdf: gpd.GeoDataFrame,
        buffer_increment=100,  # meters
        school_id_col="Code",
        school_num_students_col="Number of students",
        diss_area_id_col="DGUID",
        ):
    diss_areas_gdf = diss_areas_gdf.copy()
    schools_gdf["matched_diss_area_ids"] = ""

    # Process each school
    for idx, school in tqdm(schools_gdf.iterrows()):
        school_point = school.geometry
        target_students = school[school_num_students_col]
        matched_students = 0
        current_buffer = buffer_increment

        while matched_students < target_students:
            # Create buffer around school
            buffer_area = school_point.buffer(current_buffer)

            # Find diss_areas that intersect with buffer
            nearby_diss_areas = diss_areas_gdf[
                diss_areas_gdf.intersects(buffer_area)
            ].copy()

            # Calculate the number of secondary students in matched dissemination areas
            # The logic assumes that all ages from 10-19 years are equally divided.
            col_age_10_14 = "Total - Age groups of the population - 100% data > 0 to 14 years > 10 to 14 years"
            col_age_15_19 = "Total - Age groups of the population - 100% data > 15 to 64 years > 15 to 19 years"
            matched_students = (nearby_diss_areas[col_age_10_14].sum() + nearby_diss_areas[col_age_15_19].sum()) * 5 / 10  # We only want 5/10 of all kids with ages 10-19 (10), because secondary is 12-16 (5)
            current_buffer += buffer_increment

        schools_gdf.loc[idx, "matched_diss_area_ids"] = ",".join(nearby_diss_areas[diss_area_id_col].values)

    return schools_gdf

In [29]:
# Now build relationship between schools and stascan dissemination areas polygons

# Read files
schools = pd.read_csv("data/output/schools_qc_basic.csv")
diss_areas = gpd.read_parquet("data/output/census_2021_qc_parsed.parquet")
schools_gdf = gpd.GeoDataFrame(
    schools,
    geometry=[Point(lon, lat) for lon, lat in zip(schools['lon'], schools['lat'])],
    crs="EPSG:4326"  # Assuming input coordinates are in WGS84
)

# Reproject both datasets UTM
utm_crs = schools_gdf.estimate_utm_crs()
schools_gdf = schools_gdf.to_crs(utm_crs)
diss_areas_gdf = diss_areas.to_crs(utm_crs)
combined_gdf = combined_gdf.to_crs(utm_crs)

# Match
schools_with_diss_area_ids = match_schools_to_diss_areas(schools_gdf, diss_areas_gdf)
schools_with_diss_area_ids

465it [00:20, 22.72it/s]


,Code,School Name,lat,lon,Full Address,Code Postal,Aggregate Dissemination Area Code,Number of students,geometry,matched_diss_area_ids
0,106501,Jean-de-Brébeuf,45.501478,-73.623320,"3200 Chemin de la Côte-Sainte-Catherine, Montr...",H3T 1C9,24660146,1141,POINT (138810.902 5049066.129),"2021S051224661305,2021S051224661337,2021S05122..."
1,107501,Jean de la Mennais,45.412817,-73.481456,"870 Chemin de Saint-Jean, La Prairie, QC J5R 2...",J5R 2E6,24670001,1518,POINT (149343.113 5038587.280),"2021S051224670015,2021S051224670001,2021S05122..."
2,114501,Jean-Eudes,45.555353,-73.578775,"3535 Boulevard Rosemont, Montréal, QC H1X 1K7,...",H1X 1K7,24660067,1788,POINT (142632.755 5054851.409),"2021S051224661992,2021S051224662005,2021S05122..."
3,115501,Jésus-Marie de Sillery,46.778420,-71.250697,"2047 Chemin Saint-Louis, Québec, QC G1T 1S6, C...",G1T 1S6,24230052,595,POINT (328186.003 5183000.890),"2021S051224230225,2021S051224230218,2021S05122..."
4,116501,Collège Laval,45.615166,-73.649034,"1275 Avenue du Collège, Laval, QC H7C 1W8, Canada",H7C 1W8,24650007,2068,POINT (137535.361 5061812.030),"2021S051224650244,2021S051224662809,2021S05122..."
...,...,...,...,...,...,...,...,...,...,...
460,888069,Beurling,45.448551,-73.587426,"6100 Boulevard Champlain, Montréal, QC H4H 1A5...",H4H 1A5,24660215,206,POINT (141278.530 5043025.275),"2021S051224661198,2021S051224661197,2021S05122..."
461,889014,Châteauguay Valley,45.124516,-74.007909,"1597 Route 138A, Ormstown, QC J0S 1K0, Canada",J0S 1K0,24690001,616,POINT (106157.942 5008995.297),"2021S051224690058,2021S051224690056,2021S05122..."
462,89501,Esther Blondin,45.955941,-73.563103,"101 Rue Sainte Anne, Saint-Jacques, QC J0K 2R0...",J0K 2R0,24630001,1197,POINT (146400.323 5099283.974),"2021S051224630061,2021S051224630054,2021S05122..."
463,9502,Kells,45.459498,-73.631130,"6865 Boulevard De Maisonneuve Ouest, Montréal,...",H4B 1T1,24660203,135,POINT (137931.421 5044437.785),"2021S051224660743,2021S051224660868,2021S05122..."


In [ ]:
# Make a map of school areas
schools_with_diss_area_ids["geometry_school_location"] = schools_with_diss_area_ids["geometry"]
schools_with_diss_area_ids["geometry"] = None
for idx, row in tqdm(schools_with_diss_area_ids.iterrows()):
    matched_diss_area_ids = row["matched_diss_area_ids"].split(",")
    # For all diss area id, aggregate polygons into multipolygons
    polygons = [combined_gdf[combined_gdf["DGUID"]==dguid].geometry.values[0] for dguid in matched_diss_area_ids]
    multipolygon = MultiPolygon(polygons)
    schools_with_diss_area_ids.loc[idx, "geometry"] = multipolygon

schools_with_diss_area_ids.to_parquet(
    "data/output/schools_qc_with_diss_areas.parquet",
    compression="snappy",
    index=False,
    engine="pyarrow",
)
schools_with_diss_area_ids

465it [00:11, 39.61it/s]


,Code,School Name,lat,lon,Full Address,Code Postal,Aggregate Dissemination Area Code,Number of students,geometry,matched_diss_area_ids,geometry_school_location
0,106501,Jean-de-Brébeuf,45.501478,-73.623320,"3200 Chemin de la Côte-Sainte-Catherine, Montr...",H3T 1C9,24660146,1141,"MULTIPOLYGON (((138711.282 5050078.853, 138740...","2021S051224661305,2021S051224661337,2021S05122...",POINT (138810.902 5049066.129)
1,107501,Jean de la Mennais,45.412817,-73.481456,"870 Chemin de Saint-Jean, La Prairie, QC J5R 2...",J5R 2E6,24670001,1518,"MULTIPOLYGON (((148695.179 5039532.464, 148704...","2021S051224670015,2021S051224670001,2021S05122...",POINT (149343.113 5038587.280)
2,114501,Jean-Eudes,45.555353,-73.578775,"3535 Boulevard Rosemont, Montréal, QC H1X 1K7,...",H1X 1K7,24660067,1788,"MULTIPOLYGON (((142979.572 5054206.679, 142949...","2021S051224661992,2021S051224662005,2021S05122...",POINT (142632.755 5054851.409)
3,115501,Jésus-Marie de Sillery,46.778420,-71.250697,"2047 Chemin Saint-Louis, Québec, QC G1T 1S6, C...",G1T 1S6,24230052,595,"MULTIPOLYGON (((327482.954 5182253.035, 327435...","2021S051224230225,2021S051224230218,2021S05122...",POINT (328186.003 5183000.890)
4,116501,Collège Laval,45.615166,-73.649034,"1275 Avenue du Collège, Laval, QC H7C 1W8, Canada",H7C 1W8,24650007,2068,"MULTIPOLYGON (((137251.248 5061131.837, 137274...","2021S051224650244,2021S051224662809,2021S05122...",POINT (137535.361 5061812.030)
...,...,...,...,...,...,...,...,...,...,...,...
460,888069,Beurling,45.448551,-73.587426,"6100 Boulevard Champlain, Montréal, QC H4H 1A5...",H4H 1A5,24660215,206,"MULTIPOLYGON (((141781.764 5042735.891, 141524...","2021S051224661198,2021S051224661197,2021S05122...",POINT (141278.530 5043025.275)
461,889014,Châteauguay Valley,45.124516,-74.007909,"1597 Route 138A, Ormstown, QC J0S 1K0, Canada",J0S 1K0,24690001,616,"MULTIPOLYGON (((110610.754 5010322.506, 110568...","2021S051224690058,2021S051224690056,2021S05122...",POINT (106157.942 5008995.297)
462,89501,Esther Blondin,45.955941,-73.563103,"101 Rue Sainte Anne, Saint-Jacques, QC J0K 2R0...",J0K 2R0,24630001,1197,"MULTIPOLYGON (((153422.561 5096992.513, 153435...","2021S051224630061,2021S051224630054,2021S05122...",POINT (146400.323 5099283.974)
463,9502,Kells,45.459498,-73.631130,"6865 Boulevard De Maisonneuve Ouest, Montréal,...",H4B 1T1,24660203,135,"MULTIPOLYGON (((138109.015 5044542.351, 137953...","2021S051224660743,2021S051224660868,2021S05122...",POINT (137931.421 5044437.785)
